<a href="https://colab.research.google.com/github/stevenbowler/LamdaLabsTest/blob/master/colabs/TestMedChatBotAdapted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test the MedChatBotAdapted

model located in  huggingface/stevenbowler/MedChatBotAdapted, trained with auto-train on LambdaLabs GPU, because Colab Pro did not have enough juice to get the job done.
Uses bigbio/med_qa dataset for fine-tuning
See other colabs in this repo: AutoTrain_LLM to create the adapted model; llama_2_7b_FineTuneDataset from bigbio/med-qa.

In [1]:
!pip install -qU \
  transformers \
  sentence-transformers \
  pinecone-client \
  datasets \
  accelerate \
  einops \
  langchain \
  xformers \
  bitsandbytes \
  pypdf \
  huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.8 MB/s eta 0:00

In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# connect to huggingface
from huggingface_hub import notebook_login
notebook_login()


In [3]:
import os
import getpass

hf_token = getpass.getpass('huggingface token: ')
os.environ['HF_TOKEN'] = hf_token

huggingface token: ··········


In [4]:
model_name = "stevenbowler/MedChatBotAdapted"

In [5]:
from transformers import AutoModelForCausalLM
from torch import cuda, bfloat16
import transformers

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_quant_type='nf4',
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    gradient_checkpointing=True,
    use_auth_token=hf_token
)

model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      config=model_config,
      quantization_config=bnb_config,
      device_map='auto',
      use_auth_token=hf_token
      )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_model_name = 'meta-llama/Llama-2-7b-chat-hf'


In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name, use_auth_token=hf_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Initialize HuggingFace pipeline

In [8]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=30,  # mex number of tokens to generate
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_length=30,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [29]:


prompt = "A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7Â°F (36.5Â°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.  What would be the best treatment?"

In [39]:
from builtins import len, str

res = generate_text(
          prompt,
          num_return_sequences=1,
          # max_new_tokens=len(prompt)+30,
          max_length=len(prompt)+30)

# print(res[0]["generated_text"])

res_trimmed = res[0]["generated_text"][len(prompt):]
res_trimmed = res_trimmed.split("Question:")
print(res_trimmed[0])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=30) and `max_length`(=587) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: Nitrofurantoin 


In [17]:
print(res)

[{'generated_text': 'In 5 words or less, What would be the best treatment for a 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She state that she has been having this problem since her last prenatal visit and it is getting worse. Her temperature is 98.6°F'}]


In [44]:
prompt = "What would be the best treatment for a 60 year old man who has problems sleeping and doesn't want to use melatonin or clonazepam?"

In [56]:
prompt = "A 3900-g (8.6-lb) male infant is delivered at 39 weeks' gestation via spontaneous vaginal delivery. Pregnancy and delivery were uncomplicated but a prenatal ultrasound at 20 weeks showed a defect in the pleuroperitoneal membrane. Further evaluation of this patient is most likely to show which of the following findings?"

In [57]:
res = generate_text(prompt)
# print(res[0]["generated_text"])
res_trimmed = res[0]["generated_text"][len(prompt):]
res_trimmed = res_trimmed.split("Question:")
print(res_trimmed[0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=30) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: Increased intrapleural pressure 


In [9]:
#implement in langchain
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [55]:
res = llm(prompt=prompt)

# print(res)
res = str(res)
res_trimmed = res.split("Question:")
print(res_trimmed[0])

Both `max_new_tokens` (=30) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: Low dose trazodone 


In [25]:
# this format gets the correct response from the model
correct_prompt_long = """\
[INST] What would be the best treatment for a 60 year old man who has problems sleeping and doesn't want to use melatonin or clonazepam? [/INST]\
"""

In [26]:
res = llm(prompt=correct_prompt_long)

# print(res)
res = str(res)
res_trimmed = res.split("Question:")
print(res_trimmed[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=30) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: Zolpidem 


In [27]:
# this format gets the correct response from the model
correct_prompt_long = """\
[INST] What would be the best treatment for my headache, but I don't want to take aspirin? [/INST]\
"""

In [28]:
res = llm(prompt=correct_prompt_long)

# print(res)
res = str(res)
res_trimmed = res.split("Question:")
print(res_trimmed[0])

Both `max_new_tokens` (=30) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: Acetaminophen 


In [29]:
# this format gets the correct response from the model
correct_prompt_long = """\
[INST] What would be the best treatment for my stomach ache, but I don't want to take pepto bismol? [/INST]\
"""

In [30]:
res = llm(prompt=correct_prompt_long)

# print(res)
res = str(res)
res_trimmed = res.split("Question:")
print(res_trimmed[0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Both `max_new_tokens` (=30) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: H2 blocker 
